<a href="https://colab.research.google.com/github/Ken-huea/Language-processing/blob/main/%E6%B7%BB%E5%89%8A%E8%AA%B2%E9%A1%8C_%E6%B8%85%E6%9B%B8%E7%94%A8%E2%91%A0_mecab.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

### 問題　文章のベクトル化


<font color="red">分類問題</font>です。

訓練データ：記事とラベル
　　
学習機に記事を投入したらどのラベルなのか？というのを予測するモデルを作成します

そして今回は形態素解析ツールに<font color="red">mecab</font>を用いる事とします。


- ①記事のパスを取得し、一部を読込。合わせてそれに呼応したラベルも設定。
- ②それらを訓練データとテストデータに分割
- ④トレインデータとテストデータを分かち書き(mecab)を行いつつ、ベクトルに変換
- ⑤それを元に学習。
- ⑥精度確認と未知のデータを学習機に予測させ、結果を確認。

##### **この添削問題では、データセットを使用しますので、下記コードを実行ください**

In [ ]:
# 初めの一回だけこのセルと次のセルを実行してください、データセットをダウンロードして展開します
# 一回実行すれば、データセットはダウンロードされたままなので、再起動後等再び実行する必要はありません
import urllib.request
import zipfile

# URLを指定
url = "https://storage.googleapis.com/tutor-contents-dataset/5050_nlp_data.zip"
save_name = url.split('/')[-1]

# ダウンロードする
mem = urllib.request.urlopen(url).read()

# ファイルへ保存
with open(save_name, mode='wb') as f:
    f.write(mem)

# zipファイルをカレントディレクトリに展開する
zfile = zipfile.ZipFile(save_name)
zfile.extractall('.')

In [ ]:
# !pip install Janome==0.3.7
!pip install janome
# ライブラリ読み込み　今回はジャノメ
from janome.tokenizer import Tokenizer

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


## 実装

#### ディレクトリから記事のパス取得と読み込み
- ①ディレクトリから指定記事のパスを一括取得
- ②その中からどのくらい実際に読み込むのか選択し読込。
- ③返り値をリストに格納

In [ ]:
import glob
import random
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.naive_bayes import MultinomialNB
from sklearn.ensemble import RandomForestClassifier
from janome.tokenizer import Tokenizer

抽出する記事を辞書型にして、keyにディレクトリ名、valueに番号を入力。
　
例えば「dokujo-tsushin」の記事は１番です。みたいな。

実は言うと、このリストと辞書の設定は真下のの独自関数に格納した方が良いのです。

そうしないと<font color="red">実行の連打</font>等を行うと、溜まった成果物が初期化されずに重なりますので。

例えばこれでリストの内容をリセットしている。`docs = []`　今回は可読性の為に外に出しているけど、実行の際は意識しておくこと。

In [ ]:
docs = []          # 記事内容格納リスト
labels = []        # 記事ラベル格納リスト

# ちゃんとkeyの値はディレクトリ名に則て下さい。後でそのkeyを変数名として呼び出しますので。
category = {
        'dokujo-tsushin': 1,
        'it-life-hack': 2,
        'kaden-channel': 3,
        'livedoor-homme': 4,
        'movie-enter': 5,
        'peachy': 6,
        'smax': 7,
        'sports-watch': 8,
        'topic-news': 9
    }

独自関数です。詳細は[こちらのcolab](https://colab.research.google.com/drive/1OgKT6Wl121ySOWemNRCWqjZsl21Hdicu#scrollTo=C-RgQAK_bciR)を確認してください。

In [ ]:

def load_livedoor_news_corpus():
  print(category.items())
  for category_key, category_value in category.items():
    print(category_value)
    files = glob.glob(f"./5050_nlp_data/{category_key}/{category_key}*.txt")
    text = ''
    for file in files[:30]: # 実行時間の関係上読み込むデータを制限しています。
      print(f'file:{file}')
      with open(file, 'r', errors='ignore') as f:
          lines = f.read().splitlines()
          print("Lines:",lines)
        
          subject = lines[2]
          print("subject:",subject)
          body = "".join(lines[3:])
          print("body:",body)
          text = subject + body
          print("text:",text)
          print()

      docs.append(text)
      labels.append(category_value)

  return docs, labels

docs,labels = load_livedoor_news_corpus()


dict_items([('dokujo-tsushin', 1), ('it-life-hack', 2), ('kaden-channel', 3), ('livedoor-homme', 4), ('movie-enter', 5), ('peachy', 6), ('smax', 7), ('sports-watch', 8), ('topic-news', 9)])
1
file:./5050_nlp_data/dokujo-tsushin/dokujo-tsushin-6572058.txt
Lines: ['http://news.livedoor.com/article/detail/6572058/', '2012-05-18T14:00:00+0900', '私、職場での呼び名に悩んでいます', '昨年の秋、転職した坂本エリコさん（28歳・不動産関連）の悩みは職場での呼び名。同僚たちは名前で呼び合っていたり「ちゃん付け」だったりするのだが、エリコさんだけはいつまでたっても「坂本さん」と呼ばれている。', '「同じ時期に転職した同僚は周囲から『メグちゃん』って呼ばれているんです。なのに私だけいつまでたっても『坂本さん』というのは、やっぱり職場に馴染めていないような気がして…。同僚と一緒にお昼を食べているし、飲み会にもよく誘われるので気にしなくてもいいのかもしれませんが…」とエリコさんは話してくれた。', '', '呼び名以外で特に問題が無いのなら、気にする必要もないかもしれない。ただ呼び名は人間関係の距離感が現れるものだけに、エリコさんの悩みも理解できる。', '', 'エリコさんの悩みを聞いて思い出したのが学生時代の友人・土金ムツミさん（仮名/27歳・IT関連）の話だ。', '', 'ムツミさんの職場は上司部下関係なく「さん付け」が基本となっているが、飲み会などではニックネームか「ちゃん付け」になる。 以前は飲み会でも「土金（つちがね）さん」と呼ばれていたムツミさんだが、たまたまメガネをコンタクトレンズに髪型をショートカットにしたところニックネーム呼びに変わったという。', '', '「学生時代に『ドキンちゃん』って呼ばれていたことを話したら、その日からドキンちゃんになりました。前にも飲み会でニックネームの話はしていたんですけれど「土金だからドキ

In [ ]:
labels
# len(labels)

[1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 2,
 2,
 2,
 2,
 2,
 2,
 2,
 2,
 2,
 2,
 2,
 2,
 2,
 2,
 2,
 2,
 2,
 2,
 2,
 2,
 2,
 2,
 2,
 2,
 2,
 2,
 2,
 2,
 2,
 2,
 3,
 3,
 3,
 3,
 3,
 3,
 3,
 3,
 3,
 3,
 3,
 3,
 3,
 3,
 3,
 3,
 3,
 3,
 3,
 3,
 3,
 3,
 3,
 3,
 3,
 3,
 3,
 3,
 3,
 3,
 4,
 4,
 4,
 4,
 4,
 4,
 4,
 4,
 4,
 4,
 4,
 4,
 4,
 4,
 4,
 4,
 4,
 4,
 4,
 4,
 4,
 4,
 4,
 4,
 4,
 4,
 4,
 4,
 4,
 4,
 5,
 5,
 5,
 5,
 5,
 5,
 5,
 5,
 5,
 5,
 5,
 5,
 5,
 5,
 5,
 5,
 5,
 5,
 5,
 5,
 5,
 5,
 5,
 5,
 5,
 5,
 5,
 5,
 5,
 5,
 6,
 6,
 6,
 6,
 6,
 6,
 6,
 6,
 6,
 6,
 6,
 6,
 6,
 6,
 6,
 6,
 6,
 6,
 6,
 6,
 6,
 6,
 6,
 6,
 6,
 6,
 6,
 6,
 6,
 6,
 7,
 7,
 7,
 7,
 7,
 7,
 7,
 7,
 7,
 7,
 7,
 7,
 7,
 7,
 7,
 7,
 7,
 7,
 7,
 7,
 7,
 7,
 7,
 7,
 7,
 7,
 7,
 7,
 7,
 7,
 8,
 8,
 8,
 8,
 8,
 8,
 8,
 8,
 8,
 8,
 8,
 8,
 8,
 8,
 8,
 8,
 8,
 8,
 8,
 8,
 8,
 8,
 8,
 8,
 8,
 8,
 8,
 8,
 8,
 8,
 9,
 9,
 9,
 9,
 9,
 9,
 9,
 9,
 9,
 9,


In [ ]:
docs

['私、職場での呼び名に悩んでいます昨年の秋、転職した坂本エリコさん（28歳・不動産関連）の悩みは職場での呼び名。同僚たちは名前で呼び合っていたり「ちゃん付け」だったりするのだが、エリコさんだけはいつまでたっても「坂本さん」と呼ばれている。「同じ時期に転職した同僚は周囲から『メグちゃん』って呼ばれているんです。なのに私だけいつまでたっても『坂本さん』というのは、やっぱり職場に馴染めていないような気がして…。同僚と一緒にお昼を食べているし、飲み会にもよく誘われるので気にしなくてもいいのかもしれませんが…」とエリコさんは話してくれた。呼び名以外で特に問題が無いのなら、気にする必要もないかもしれない。ただ呼び名は人間関係の距離感が現れるものだけに、エリコさんの悩みも理解できる。エリコさんの悩みを聞いて思い出したのが学生時代の友人・土金ムツミさん（仮名/27歳・IT関連）の話だ。ムツミさんの職場は上司部下関係なく「さん付け」が基本となっているが、飲み会などではニックネームか「ちゃん付け」になる。 以前は飲み会でも「土金（つちがね）さん」と呼ばれていたムツミさんだが、たまたまメガネをコンタクトレンズに髪型をショートカットにしたところニックネーム呼びに変わったという。「学生時代に『ドキンちゃん』って呼ばれていたことを話したら、その日からドキンちゃんになりました。前にも飲み会でニックネームの話はしていたんですけれど「土金だからドキンなのね」くらいの反応でまったく定着しなかったんですよ（苦笑）。とりあえず上司や同僚との距離も縮まった気がして嬉しかったです」とムツミさんは話してくれた。ところで呼び名から恋愛関係が分かってしまうことは多い。残業を終えたイクミさん（34歳・家電メーカー）が会社近くお店で食事をしていたところ、たまたまサブマネージャーのTさん（36歳）と同僚のM子さん（28歳）が、近くの席に座った。イクミさんに気づかないまま話し始めた二人…。「ウチの職場では残業後に一緒にお茶や食事をすることは珍しくないんですけど、TさんとM子は、み〜ちゃん、たあ君って呼び始めたんです。ビックリしましたよ。いつの間にそういう関係に！ って感じでした。今では職場でも公認のカップルで、オンオフでの二人の呼び名の切り替えも名物化しています（笑）」とイクミさん。身近なところで同様の体験をしている人も

#### データを訓練データと検証データに分割

In [ ]:
# 独自関数の返り値の確認
print(docs[0])
print("docsにある記事の数:",len(docs),"　docsのtype:",type(docs)) #取得記事 9*30で270
print()
print(labels[0])
print("labelsにある記事の数:",len(labels),"labelsのtype:",type(labels)) #取得記事 9*30で270

私、職場での呼び名に悩んでいます昨年の秋、転職した坂本エリコさん（28歳・不動産関連）の悩みは職場での呼び名。同僚たちは名前で呼び合っていたり「ちゃん付け」だったりするのだが、エリコさんだけはいつまでたっても「坂本さん」と呼ばれている。「同じ時期に転職した同僚は周囲から『メグちゃん』って呼ばれているんです。なのに私だけいつまでたっても『坂本さん』というのは、やっぱり職場に馴染めていないような気がして…。同僚と一緒にお昼を食べているし、飲み会にもよく誘われるので気にしなくてもいいのかもしれませんが…」とエリコさんは話してくれた。呼び名以外で特に問題が無いのなら、気にする必要もないかもしれない。ただ呼び名は人間関係の距離感が現れるものだけに、エリコさんの悩みも理解できる。エリコさんの悩みを聞いて思い出したのが学生時代の友人・土金ムツミさん（仮名/27歳・IT関連）の話だ。ムツミさんの職場は上司部下関係なく「さん付け」が基本となっているが、飲み会などではニックネームか「ちゃん付け」になる。 以前は飲み会でも「土金（つちがね）さん」と呼ばれていたムツミさんだが、たまたまメガネをコンタクトレンズに髪型をショートカットにしたところニックネーム呼びに変わったという。「学生時代に『ドキンちゃん』って呼ばれていたことを話したら、その日からドキンちゃんになりました。前にも飲み会でニックネームの話はしていたんですけれど「土金だからドキンなのね」くらいの反応でまったく定着しなかったんですよ（苦笑）。とりあえず上司や同僚との距離も縮まった気がして嬉しかったです」とムツミさんは話してくれた。ところで呼び名から恋愛関係が分かってしまうことは多い。残業を終えたイクミさん（34歳・家電メーカー）が会社近くお店で食事をしていたところ、たまたまサブマネージャーのTさん（36歳）と同僚のM子さん（28歳）が、近くの席に座った。イクミさんに気づかないまま話し始めた二人…。「ウチの職場では残業後に一緒にお茶や食事をすることは珍しくないんですけど、TさんとM子は、み〜ちゃん、たあ君って呼び始めたんです。ビックリしましたよ。いつの間にそういう関係に！ って感じでした。今では職場でも公認のカップルで、オンオフでの二人の呼び名の切り替えも名物化しています（笑）」とイクミさん。身近なところで同様の体験をしている人も多い

データ分割

<font color="red">train_test_split</font> を行います。　この分割方法を用いればshffleする手間を省略できますので。

しかしながら、上記のように<font color="red">docsとlabelsがlist</font>なので、そこを調整する必要がありそうです。

In [ ]:
import numpy as np
from sklearn.model_selection import train_test_split

# リストからnumpyへ。 
docs2 = np.array(docs)
labels2 = np.array(labels)

# データ分割 比率は9:1
x_train , x_test,y_train,y_test = train_test_split(docs2,labels2, test_size=0.1)

# shapeも確認
x_train.shape , x_test.shape,y_train.shape,y_test.shape

# lebelsはone hot encording しなくても平気です。別に deeplearnig ではないので。 

((243,), (27,), (243,), (27,))

###### train_test_split以外の方法
train_test_split以外の分割方法を用いる場合はこのようにシャッフル処理の手間が必要です。

- docsのlenの数だけ値を生成しています。(記事の数分)
- その数だけリストを生成して indicesという変数に格納しております。
- それをランダムに並び替えています。

In [ ]:
# indices は0からドキュメントの数までの整数をランダムに並べ替えた配列
# list(range(len(docs)))はデータの長さそれをリストにしている。 docsのlenの数だけのリストを生成して、indicesに格納している。
random.seed()
indices = list(range(len(docs)))

random.shuffle(indices)

print(type(indices))     # <class 'list'>
print(len(indices))      # 27
print(indices)           # [4, 16, 5, 26, 13, 7, 1, 9, 23, 12, 19, 11, 25, 22, 14, 6, 0, 21, 8, 17, 10, 24, 18, 15, 2, 3, 20]


<class 'list'>
270
[105, 77, 243, 98, 168, 130, 24, 238, 157, 171, 162, 120, 260, 29, 58, 213, 242, 212, 178, 228, 102, 107, 121, 93, 269, 174, 161, 45, 133, 106, 95, 27, 233, 63, 179, 48, 40, 6, 245, 110, 197, 224, 125, 198, 109, 28, 185, 186, 55, 30, 43, 68, 87, 59, 84, 147, 142, 4, 113, 56, 200, 2, 53, 112, 61, 14, 137, 7, 258, 52, 188, 69, 150, 176, 209, 124, 99, 194, 5, 165, 108, 183, 177, 175, 255, 163, 25, 38, 261, 101, 259, 146, 126, 17, 132, 154, 100, 74, 16, 149, 184, 46, 49, 67, 22, 221, 151, 9, 31, 218, 131, 220, 13, 128, 138, 264, 152, 173, 1, 136, 207, 166, 262, 210, 148, 230, 182, 253, 172, 73, 246, 267, 41, 88, 34, 33, 266, 47, 239, 37, 8, 117, 195, 226, 85, 135, 26, 86, 103, 193, 217, 169, 15, 140, 203, 116, 234, 75, 159, 181, 91, 237, 122, 44, 143, 251, 153, 247, 11, 71, 114, 90, 97, 104, 189, 214, 155, 225, 241, 216, 204, 206, 18, 235, 129, 160, 139, 158, 265, 229, 215, 249, 156, 223, 199, 180, 187, 256, 96, 72, 134, 21, 127, 191, 115, 94, 66, 145, 141, 10, 92, 111, 

#### 文章のベクトル化

In [ ]:
# 全パートの確認
x_train.shape , x_test.shape,y_train.shape,y_test.shape

((243,), (27,), (243,), (27,))

分かち書きを行う関数。

In [ ]:
# メカブのインポート

!apt install aptitude
!aptitude install mecab libmecab-dev mecab-ipadic-utf8 git make curl xz-utils file -y
!pip install mecab-python3==0.7

Reading package lists... Done
Building dependency tree       
Reading state information... Done
aptitude is already the newest version (0.8.10-6ubuntu1).
0 upgraded, 0 newly installed, 0 to remove and 20 not upgraded.
mecab is already installed at the requested version (0.996-5)
libmecab-dev is already installed at the requested version (0.996-5)
mecab-ipadic-utf8 is already installed at the requested version (2.7.0-20070801+main-1)
git is already installed at the requested version (1:2.17.1-1ubuntu0.12)
make is already installed at the requested version (4.1-9.1ubuntu1)
curl is already installed at the requested version (7.58.0-2ubuntu3.19)
xz-utils is already installed at the requested version (5.2.2-1.3ubuntu0.1)
file is already installed at the requested version (1:5.32-2ubuntu0.4)
mecab is already installed at the requested version (0.996-5)
libmecab-dev is already installed at the requested version (0.996-5)
mecab-ipadic-utf8 is already installed at the requested version (2.7.0-2

In [ ]:
# ここで分かち書きの設定。
import MeCab
mecab = MeCab.Tagger('-Owakati')

In [ ]:
# 分かち書きの独自関数
def tokenize1(text):
    Keitaiso = []
    tokens = mecab.parse(text)
    tokens = tokens.split(' ')
    for token in tokens:
        Keitaiso.append(token)            
    return Keitaiso

In [ ]:
print(x_train[0])      # x_trainの要素の一部を確認
tokenize1(x_train[0])  # 同一のx_trainの要素の一部を上記の独自関数にかけてみたらどうなるのかを確認。

梅村ゆずの結婚どころじゃないんですっ！vol.2　Presented by ゆるっとcafeどうも、花の独身エロ漫画家・梅村ゆず（33）です。いきなりですが、独女の皆さん、姪や甥と仲いいですか？先日、私の姪（17歳）が美容師になり、一人暮らしをするとのことで、 実家でささやかな就職祝いを開きました。 餃子、おにぎり、じゃがいも煮という、本当にささやかなパーティー。2Kの狭い我が家に来たのは、長兄家族の姪2人（12歳＆17歳）、次男家族の甥2人（9歳＆12歳）と、次男嫁(39歳）。 我が家は、さほど息子家族とも交流に力を入れないので姪と甥が揃うなんて3年に1回もないぐらい。でも、さすが同世代が集まると、仲良くなるのがめちゃくちゃ早い。オセロをやったり、トランプやったりと、賑やかに過ごしてました。 途中、そんな独女の私が会話に入るも、どうにもズレてしまうのです。姪と甥が仲いいので「つきあっちゃえば？」なんてジョークで言ったら、一瞬でしーんとなりました。いったい何を言ったら、平成生まれの彼女達は笑ってくれるのだろう…？叔母である私は、どうにも彼等のテンションになじめず、後半はほとんど隣の部屋に逃げ隠れてました。気づけば父と母もおらず。次男嫁はちゃんとついていって、トランプも最後まで参加していました。この年になると、血がつながっている姪甥といえど、ヤングとの接し方がわからない。そのテンション、感受性、会話。ジェネレーションギャップが埋まりません！とはいえ、つながっていないはずの次男嫁はちゃんとついていけていて、最後は姪とメアド交換までしてました。子を持つ人と持たない人との差だと思いたいけど、果たしてそれだけなのかしら？家にこもって仕事ばっかりしてるから、人と話すことが苦手になったのかな？同じ人とばかり話すし。そういえばこの頃、新しい友人ってできないなぁ…。独女の皆さんはいかがですか？ おなかの中にいたときから知っているので、姪が就職なんてなんだか不思議な感じです。 結婚も出産も、先を越されるんじゃなかろうか？  あぁ、姪甥に誇れるものなんて何もない叔母です。まぁ、結局は大人同士で、先行き見えない仕事の話とか恋バナしてるほうが私には合っているのかも。ん？これも「負け犬の遠吠え」ってやつになるのでしょうか？ワオーンワオーン！！


['梅村',
 'ゆず',
 'の',
 '結婚',
 'どころ',
 'じゃ',
 'ない',
 'んで',
 'すっ',
 '！',
 'vol',
 '.',
 '2',
 '\u3000',
 'Presented',
 'by',
 'ゆる',
 'っと',
 'cafe',
 'どうも',
 '、',
 '花',
 'の',
 '独身',
 'エロ',
 '漫画',
 '家',
 '・',
 '梅村',
 'ゆず',
 '（',
 '33',
 '）',
 'です',
 '。',
 'いきなり',
 'です',
 'が',
 '、',
 '独',
 '女',
 'の',
 '皆さん',
 '、',
 '姪',
 'や',
 '甥',
 'と',
 '仲',
 'いい',
 'です',
 'か',
 '？',
 '先日',
 '、',
 '私',
 'の',
 '姪',
 '（',
 '17',
 '歳',
 '）',
 'が',
 '美容',
 '師',
 'に',
 'なり',
 '、',
 '一人暮らし',
 'を',
 'する',
 'と',
 'の',
 'こと',
 'で',
 '、',
 '実家',
 'で',
 'ささやか',
 'な',
 '就職',
 '祝い',
 'を',
 '開き',
 'まし',
 'た',
 '。',
 '餃子',
 '、',
 'おにぎり',
 '、',
 'じゃがいも',
 '煮',
 'という',
 '、',
 '本当に',
 'ささやか',
 'な',
 'パーティー',
 '。',
 '2',
 'K',
 'の',
 '狭い',
 '我が家',
 'に',
 '来',
 'た',
 'の',
 'は',
 '、',
 '長兄',
 '家族',
 'の',
 '姪',
 '2',
 '人',
 '（',
 '12',
 '歳',
 '＆',
 '17',
 '歳',
 '）',
 '、',
 '次男',
 '家族',
 'の',
 '甥',
 '2',
 '人',
 '（',
 '9',
 '歳',
 '＆',
 '12',
 '歳',
 '）',
 'と',
 '、',
 '次男',
 '嫁',
 '(',
 '39',
 '歳',
 '）',
 '。',
 '我が家',
 'は',
 '、',
 '

Tf-idfを用いてをベクトル化を行う。

In [ ]:
# ベクトル表現化を行う変換器を生成。　引数の「tokenizer=tokenize1」。このvectorizerに組み込ませる文章はtokenize1 という独自関数にかけますよ。⇒ 分かち書きされますよ。
vectorizer = TfidfVectorizer(tokenizer=tokenize1)

訓練データの場合は fit_transform関数 を用る。

検証データの場合は,訓練データの fit() の結果に基づくので、 transform()関数 を行う必要があります。　⇒[こちら](https://premium.aidemy.jp/courses/5050/exercises/B1fusDDtef)

In [ ]:
#　ベクトルに変換⇒fitは学習する
x_train_matrix = vectorizer.fit_transform(x_train)  # fit及びtransform  fit で学習させています。これにより｢どの単語はどのようにベクトル化するか｣を学習しております。
x_test_matrix = vectorizer.transform(x_test)        # transformのみ。　その上記の学習に則って x_test をベクトル化しています。　そしてテストデータはfitしてはダメでしょう。

#### 学習。

ベクトル化した文章(x_train_matrix)を元に分類予測の為に学習。

In [ ]:
# ナイーブベイズを用いて分類をおこなってください。
clf = MultinomialNB()
clf.fit(x_train_matrix, y_train)

# ランダムフォレストを用いて分類をおこなってください
clf2 = RandomForestClassifier()
clf2.fit(x_train_matrix, y_train)

RandomForestClassifier()

#### 精度評価

In [ ]:
# 結果を表示
print(clf.score(x_train_matrix, y_train))
print(clf.score(x_test_matrix, y_test))

print(clf2.score(x_train_matrix, y_train))
print(clf2.score(x_test_matrix, y_test))

0.9876543209876543
0.7777777777777778
1.0
0.8518518518518519


これでどの記事がどのラベルに属するのかまあまあの精度で予測できる。　という事が分かりました。

#### 実際の確認
[ここでエラー](https://aidemy-communication.slack.com/archives/C03ANT80SAE/p1662376146173589)

実際にデータを読込んで、学習機に予測させてみましょう。

今回は<font color="red">xmax(label:7)</font>から適当にパスを拾い、読み込みました。読み込んだ内容は<font color="red">読み込み時の独自関数のコード</font>を拝借。

In [ ]:
with open("/content/5050_nlp_data/smax/smax-6606357.txt", 'r', errors='ignore') as f:
          lines = f.read().splitlines()
          print("Lines:",lines)
          # 1,2行目に書いたあるURLと時間は関係ないので取り除きます。
          url = lines[0]
          print("url:",url)
          
          datetime = lines[1]
          print("datetime:",datetime)
          subject = lines[2]
          print("subject:",subject)
          body = "".join(lines[3:])
          print("body:",body)
          text = subject + body
          print("text:",text)
          print()

Lines: ['http://news.livedoor.com/article/detail/6606357/', '2012-05-29T18:35:00+0900', 'ウィルコム初Androidスマートフォン！PHSと3Gにデュアル対応した「DIGNO DUAL WX04K」を写真と動画で紹介【レポート】', 'ウィルコム初のAndroidスマートフォンを探る！ ', '', '既報通り、ウィルコムとソフトバンクモバイルは29日、2012年夏以降に発売する予定の新モデル「ソフトバンクモバイル・ウィルコム新商品発表会 2012 Summer」を開催し、音声通話はPHS＋3Gによる通話と3Gによるデータ通信に対応したデュアルネットワークなAndroidスマートフォン「DIGNO DUAL WX04K」が発表されました。発売は2012年6月21日予定。', '', 'DIGNO DUAL WX04Kは従来のようなPHS端末とは違い、「通話の選択肢にPHSを選んで利用することができる3Gスマートフォン」ともいうべき端末で、データ通信に絡む機能は他キャリアなどで提供されている3G通信のスマートフォンと同様で、音声通話にPHSを利用することでPHS（ウィルコム）の強みであるクリアな音声品質や「誰とでも定額」の恩恵を受ける事が出来るスマートフォンとなっています。', '', '発表会場にて実機の展示がありましたので紹介します。', '', '■デュアルだけど実はトリ！？', '“DIGNO DUAL”というペットネームではありますが、通話にはPHSモード・3Gモードの他にGSMにも対応したモードが用意されており、実はトライネットワーク機であったりします。', '', '', 'DIGNO DUAL WX04K', '', '本体カラーはホワイト・ピンク・グリーン・ブラックの4色', '', 'サイズは、大きさが約130×64×10.7mm（最薄部）、質量が約148g。（バッテリー装着時）OSにはAndroid 2.3（開発コード名：Gingerbread)を採用していますが、発売後にAndroid 4.0（開発コード名：IceCream Sandwich；ICS）へのアップデート予定がとのことです。', '', 'CPUは1.2GHzデュアルコアのルネサス製「MP522

In [ ]:
# 上記の内容からtextを用いれば良さそうですね。
text

'ウィルコム初Androidスマートフォン！PHSと3Gにデュアル対応した「DIGNO DUAL WX04K」を写真と動画で紹介【レポート】ウィルコム初のAndroidスマートフォンを探る！ 既報通り、ウィルコムとソフトバンクモバイルは29日、2012年夏以降に発売する予定の新モデル「ソフトバンクモバイル・ウィルコム新商品発表会 2012 Summer」を開催し、音声通話はPHS＋3Gによる通話と3Gによるデータ通信に対応したデュアルネットワークなAndroidスマートフォン「DIGNO DUAL WX04K」が発表されました。発売は2012年6月21日予定。DIGNO DUAL WX04Kは従来のようなPHS端末とは違い、「通話の選択肢にPHSを選んで利用することができる3Gスマートフォン」ともいうべき端末で、データ通信に絡む機能は他キャリアなどで提供されている3G通信のスマートフォンと同様で、音声通話にPHSを利用することでPHS（ウィルコム）の強みであるクリアな音声品質や「誰とでも定額」の恩恵を受ける事が出来るスマートフォンとなっています。発表会場にて実機の展示がありましたので紹介します。■デュアルだけど実はトリ！？“DIGNO DUAL”というペットネームではありますが、通話にはPHSモード・3Gモードの他にGSMにも対応したモードが用意されており、実はトライネットワーク機であったりします。DIGNO DUAL WX04K本体カラーはホワイト・ピンク・グリーン・ブラックの4色サイズは、大きさが約130×64×10.7mm（最薄部）、質量が約148g。（バッテリー装着時）OSにはAndroid 2.3（開発コード名：Gingerbread)を採用していますが、発売後にAndroid 4.0（開発コード名：IceCream Sandwich；ICS）へのアップデート予定がとのことです。CPUは1.2GHzデュアルコアのルネサス製「MP5225」を採用。本体メモリは2GB、RAMは1GBとなっています。この他にもIPX5、IPX7相当の防水機能を備えています。表面と裏面表側は機能・ホーム・バック・検索の4キー配置となっている他、スピーカーや近接・照度センサーに約30万画素CMOSのインカメラを備えています。裏面には約500万画素CMOSのアウトカメラに赤外線通信ユ

In [ ]:
# このtextをまずベクトル化します。
text_matrix = vectorizer.transform([text])
text_matrix

<1x14115 sparse matrix of type '<class 'numpy.float64'>'
	with 269 stored elements in Compressed Sparse Row format>

In [ ]:
# そして予測させましょう。
predict = clf2.predict(text_matrix)
predict
# array([7])

array([7])